<a href="https://colab.research.google.com/github/slayergod/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [3]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-03-17 22:37:41--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.88.109
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.88.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  96.0MB/s    in 2.0s    

2020-03-17 22:37:43 (96.0 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [4]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [5]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [6]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####

In [0]:
import pandas as pd
import numpy as np

In [14]:
opp_df = pd.read_csv('order_products__prior.csv')
print(opp_df.shape)
opp_df.sample(3)

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
6398816,675323,13166,7,1
23986686,2529763,19508,5,1
8230434,868809,17341,8,1


In [13]:
opt_df = pd.read_csv('order_products__train.csv')
print(opt_df.shape)
opt_df.sample(3)

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
613386,1510024,41920,20,0
813083,2005749,39139,13,0
451673,1105650,47209,2,1


In [15]:
orders_df = pd.read_csv('orders.csv')
print(orders_df.shape)
orders_df.sample(3)

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
600480,2685358,36218,prior,7,3,10,7.0
508162,2816184,30658,prior,11,5,7,16.0
186057,1845743,11236,prior,54,6,7,3.0


In [17]:
products_df = pd.read_csv('products.csv')
print(products_df.shape)
products_df.sample(3)

(49688, 4)


,product_id,product_name,aisle_id,department_id
8654,8655,Waxed Floss,20,11
45454,45455,Powder Anti-Perspirant Deodorant,80,11
12321,12322,"Extra Sweet, Extra Creamy Coffee Creamer",53,16


In [18]:
order_products_df = pd.concat([opp_df,opt_df])
print(order_products_df.shape)
order_products_df.sample(3)

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
17529946,1849016,13176,10,1
767614,81255,42282,6,0
22721836,2396611,32537,6,0


In [19]:
products_df.loc[products_df['product_name'] == 'Banana']

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [20]:
products_df.loc[products_df['product_name'] == 'Bag of Organic Bananas']

,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4


In [21]:
products_df.loc[products_df['product_name'] == 'Organic Strawberries']

,product_id,product_name,aisle_id,department_id
21136,21137,Organic Strawberries,24,4


In [22]:
products_df.loc[products_df['product_name'] == 'Organic Baby Spinach']

,product_id,product_name,aisle_id,department_id
21902,21903,Organic Baby Spinach,123,4


In [23]:
products_df.loc[products_df['product_name'] == 'Organic Hass Avocado']

,product_id,product_name,aisle_id,department_id
47208,47209,Organic Hass Avocado,24,4


In [24]:
products_df.loc[products_df['product_name'] == 'Organic Avocado']

,product_id,product_name,aisle_id,department_id
47765,47766,Organic Avocado,24,4


In [25]:
products_df.loc[products_df['product_name'] == 'Large Lemon']

,product_id,product_name,aisle_id,department_id
47625,47626,Large Lemon,24,4


In [26]:
products_df.loc[products_df['product_name'] == 'Strawberries']

,product_id,product_name,aisle_id,department_id
16796,16797,Strawberries,24,4


In [27]:
products_df.loc[products_df['product_name'] == 'Limes']

,product_id,product_name,aisle_id,department_id
26208,26209,Limes,24,4


In [28]:
products_df.loc[products_df['product_name'] == 'Organic Whole Milk']

,product_id,product_name,aisle_id,department_id
27844,27845,Organic Whole Milk,84,16


In [29]:
top_id = [24852,13176,21137,21903,47209,47766,47626,16797,26209,27845]
top_df = products_df[products_df['product_id'].isin(top_id) == True]
top_df.head(11)

,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4
16796,16797,Strawberries,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4
24851,24852,Banana,24,4
26208,26209,Limes,24,4
27844,27845,Organic Whole Milk,84,16
47208,47209,Organic Hass Avocado,24,4
47625,47626,Large Lemon,24,4
47765,47766,Organic Avocado,24,4


In [31]:
main_products_df = pd.merge(top_df,order_products_df, how = 'inner', on = 'product_id')
main_products_df.sample(3)

,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered
2111285,47626,Large Lemon,24,4,853723,3,1
1700354,26209,Limes,24,4,3343404,7,0
1565088,26209,Limes,24,4,53690,5,1


In [43]:
banana_df = main_products_df[main_products_df['product_id'] == (24852)]
banana_total = len(banana_df.index)
print(banana_total)

491291


In [0]:
def ordered(item_id):
  temp_df = main_products_df[main_products_df['product_id']==(item_id)]
  total = len(temp_df.index)
  return total

In [52]:
print('Bananas')
ordered(24852)

Bananas


491291

In [0]:
total_sales = [ordered(id) for id in (top_id)]

In [57]:
product_key = {'Product_Name' : ['Banana', 'Bag of Organic Bananas', 'Organic Strawberries', 'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Avocado', 
                         'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk'], 'Total_Ordered' : total_sales}
product_key                       

{'Product_Name': ['Banana',
  'Bag of Organic Bananas',
  'Organic Strawberries',
  'Organic Baby Spinach',
  'Organic Hass Avocado',
  'Organic Avocado',
  'Large Lemon',
  'Strawberries',
  'Limes',
  'Organic Whole Milk'],
 'Total_Ordered': [491291,
  394930,
  275577,
  251705,
  220877,
  184224,
  160792,
  149445,
  146660,
  142813]}

In [58]:
total_ordered_df = pd.DataFrame(product_key)
total_ordered_df

,Product_Name,Total_Ordered
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877
5,Organic Avocado,184224
6,Large Lemon,160792
7,Strawberries,149445
8,Limes,146660
9,Organic Whole Milk,142813


## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [77]:
# table2 -> tidy
tidy_table2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe','Mary Johnson'])
tidy_table2 = tidy_table2.rename(columns={
    'index':'Trt',
    'variable':'Names',
    'value':'Result'
})
tidy_table2

,Trt,Names,Result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [78]:
tidy_table2['Trt'] = tidy_table2['Trt'].str.replace('treatment', '')
tidy_table2

,Trt,Names,Result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


In [79]:
# tidy -> table2
wide_table2 = tidy_table2.pivot_table(index='Trt', columns='Names', values='Result')
wide_table2

Names,Jane Doe,John Smith,Mary Johnson
Trt,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


In [0]:
flights = sns.load_dataset('flights')

In [81]:
# Flights Pivot Table
flights.sample(3)

,year,month,passengers
64,1954,May,234
85,1956,February,277
6,1949,July,148


In [82]:
wide_flights = flights.pivot_table(index = 'year', columns = 'month', values = 'passengers')
wide_flights

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####